In [57]:
import xarray as xr
import pandas as pd
import numpy as np
import meitner
import warnings

import matplotlib.pyplot as plt

from vamas import Vamas

In [58]:
file = 'sh_all.vms'

In [59]:
data0 = meitner.Xps.from_vamas(path=file, region_id=6)
data1 = meitner.Xps.from_vamas(path=file, region_id=14)
xps_list = [data0,data1]

Found 43 blocks.

Detected multiple VAMAS blocks with the same name.

Excitation energy:
1486.69 eV
Found 43 blocks.

Detected multiple VAMAS blocks with the same name.

Excitation energy:
1486.69 eV


In [63]:
fit = meitner.Fit(xps_list, n_peaks=[4,6], first_peak_index=1)

c:\Users\Isaac\Documents\GitHub\meitner\meitner.py:265: RuntimeWarning: invalid value encountered in double_scalars
  return fl/2 + np.sqrt(np.power(fl,2)/4 + np.power(fg,2))


In [64]:
fit.params

name,value,initial value,min,max,vary,expression
d0_p1_center,-inf,None,-inf,inf,True,
d0_p1_amplitude,-inf,None,-inf,inf,True,
d0_p1_sigma,-inf,None,-inf,inf,True,
d0_p1_gamma,-inf,None,-inf,inf,True,
d0_p1_fwhm,nan,None,-inf,inf,False,"calculate_voigt_fwhm(d0_p1_sigma,d0_p1_gamma)"
d0_p2_center,-inf,None,-inf,inf,True,
d0_p2_amplitude,-inf,None,-inf,inf,True,
d0_p2_sigma,-inf,None,-inf,inf,True,
d0_p2_gamma,-inf,None,-inf,inf,True,
d0_p2_fwhm,nan,None,-inf,inf,False,"calculate_voigt_fwhm(d0_p2_sigma,d0_p2_gamma)"


In [62]:
fit.xps['d0'].ds

<xarray.Dataset>
Dimensions:         (be: 201)
Coordinates:
  * be              (be) float64 459.0 458.9 458.8 458.7 ... 439.2 439.1 439.0
Data variables:
    cps             (be) float64 2.356e+04 2.357e+04 ... 1.062e+04 1.064e+04
    ke              (be) float64 1.028e+03 1.028e+03 ... 1.048e+03 1.048e+03
    bg              (be) float64 2.361e+04 2.361e+04 ... 1.058e+04 1.058e+04
    cps_no_bg       (be) float64 -51.41 -41.57 -56.8 139.8 ... -102.4 44.77 58.0
    bg_norm         (be) float64 0.06994 0.06994 0.06994 ... 0.03479 0.03479
    cps_norm        (be) float64 0.0698 0.06982 0.06978 ... 0.03491 0.03495
    cps_no_bg_norm  (be) float64 0.006116 0.006142 ... 0.006375 0.006411